# Compare Constants

This Notebook accesses values from the xml record library and generates bokeh plots for the data.

#### Library imports

In [1]:
#Standard Python libraries
import os

#http://www.numpy.org/
import numpy as np

#http://pandas.pydata.org/
import pandas as pd

#https://github.com/usnistgov/iprPy
import iprPy

#https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

#http://bokeh.pydata.org/
from bokeh.io import show, output_notebook, save, vplot
from bokeh.plotting import figure
from bokeh.resources import Resources
output_notebook()

Loading BokehJS ...

## Input parameters

#### XML record library path

In [2]:
record_dir = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\library_2016_10_26'

#### Record search delimiters

In [3]:
calc_type = ['refine_structure_static', 'LAMMPS_ELASTIC', 'dynamic_relax']

In [4]:
symbols =   'Al-Ni'
prototype = 'B2--CsCl'

In [5]:
symbols =   'Fe'
prototype = 'A2--W--bcc'

In [6]:
symbols =   'Al'
prototype = 'A1--Cu--fcc'

## Record parsing functions

#### Composite function

In [7]:
def parse_record(model):
    """
    Parses DataModelDict record and returns a flattened dictionary representation.
    
    Current formats supported:
    - calculation-system-relax        
    """
    if   model.keys()[0] == 'calculation-system-relax':  return parse_system_relax(model)
    elif model.keys()[0] == 'calculation-dynamic-relax': return parse_dynamic_relax(model)
    else: raise ValueError('Unsupported record format')

#### calculation-system-model

In [8]:
def parse_system_relax(model):
    """Parses a calculation-system-relax DataModelDict record and returns a flattened dictionary representation."""
    
    calc = model['calculation-system-relax']
    params = {}
    params['calc_key'] =      calc['calculation']['id']
    params['calc_id'] =       calc['calculation']['script']
    params['strain_range'] =  calc['calculation']['run-parameter']['strain-range']
    params['load_options'] =  calc['calculation']['run-parameter']['load_options']
    params['size_mults'] =    calc['calculation']['run-parameter']['size-multipliers']
    params['size_mults'] =    np.array([params['size_mults']['a'], 
                                        params['size_mults']['b'], 
                                        params['size_mults']['c']])
    params['potential_key'] = calc['potential']['key']
    params['potential_id'] =  calc['potential']['id']
    params['load'] =          '%s %s' % (calc['system-info']['artifact']['format'],
                                         calc['system-info']['artifact']['file'])
    params['prototype'] =     calc['system-info']['artifact']['family']
    params['symbols'] =       calc['system-info']['symbols']
    params['temperature'] =   calc['phase-state']['temperature']['value']
    params['pressure_xx'] =   uc.value_unit(calc['phase-state']['pressure-xx'])
    params['pressure_yy'] =   uc.value_unit(calc['phase-state']['pressure-yy'])
    params['pressure_zz'] =   uc.value_unit(calc['phase-state']['pressure-zz'])
    
    try:    init = calc['as-constructed-atomic-system']
    except: pass
    else:
        params['initial_a'] = uc.value_unit(init.find('a'))
        try:    params['initial_b'] = uc.value_unit(init.find('b'))
        except: params['initial_b'] = params['initial_a']
        try:    params['initial_c'] = uc.value_unit(init.find('c'))
        except: params['initial_c'] = params['initial_a']
            
    try:    final = calc['relaxed-atomic-system']
    except: pass
    else:
        params['final_a'] = uc.value_unit(final.find('a'))
        try:    params['final_b'] = uc.value_unit(final.find('b'))
        except: params['final_b'] = params['final_a']
        try:    params['final_c'] = uc.value_unit(final.find('c'))
        except: params['final_c'] = params['final_a']    
        params['E_cohesive'] = uc.value_unit(calc['cohesive-energy'])    
        params['C'] = am.ElasticConstants(model=calc)
        
    return params   

#### calculation-dynamic-relax

In [9]:
def parse_dynamic_relax(model):
    """Parses a calculation-system-relax DataModelDict record and returns a flattened dictionary representation."""
    
    calc = model['calculation-dynamic-relax']
    params = {}
    params['calc_key'] =      calc['calculation']['id']
    params['calc_id'] =       calc['calculation']['script']
    params['load_options'] =  calc['calculation']['run-parameter']['load_options']
    params['size_mults'] =    calc['calculation']['run-parameter']['size-multipliers']
    params['size_mults'] =    np.array([params['size_mults']['a'], 
                                        params['size_mults']['b'], 
                                        params['size_mults']['c']])
    params['potential_key'] = calc['potential']['key']
    params['potential_id'] =  calc['potential']['id']
    params['load'] =          '%s %s' % (calc['system-info']['artifact']['format'],
                                         calc['system-info']['artifact']['file'])
    params['prototype'] =     calc['system-info']['artifact']['family']
    params['symbols'] =       calc['system-info']['symbols']
    params['temperature'] =   calc['phase-state']['temperature']['value']
    params['pressure_xx'] =   uc.value_unit(calc['phase-state']['pressure-xx'])
    params['pressure_yy'] =   uc.value_unit(calc['phase-state']['pressure-yy'])
    params['pressure_zz'] =   uc.value_unit(calc['phase-state']['pressure-zz'])
            
    try:    equil = calc['equilibrium-averages']
    except: pass
    else:
        params['final_a'] = uc.value_unit(equil['a'])
        params['final_a_err'] = uc.set_in_units(equil['a']['error'], equil['a']['unit'])
        
        params['final_b'] = uc.value_unit(equil['b'])
        params['final_b_err'] = uc.set_in_units(equil['b']['error'], equil['b']['unit'])
        
        params['final_c'] = uc.value_unit(equil['c'])
        params['final_c_err'] = uc.set_in_units(equil['c']['error'], equil['c']['unit'])
        
        params['E_cohesive'] = uc.value_unit(equil['cohesive-energy'])
        params['E_cohesive_err'] = uc.set_in_units(equil['cohesive-energy']['error'], equil['c']['unit'])
        
    return params   

## Read records and build pandas DataFrame

In [10]:
data = []
for record in iprPy.prepare.read_records(record_dir, symbols=symbols, prototype=prototype, calc_type=calc_type):
    
    data.append(parse_record(record['model']))
data = pd.DataFrame(data)
data

,C,E_cohesive,E_cohesive_err,calc_id,calc_key,final_a,final_a_err,final_b,final_b_err,final_c,...,potential_id,potential_key,pressure_xx,pressure_yy,pressure_zz,prototype,size_mults,strain_range,symbols,temperature
0,NaN,NaN,NaN,calc_dynamic_relax,2e29691f-46b9-44df-9d8e-ea3524f766c9,NaN,NaN,NaN,NaN,NaN,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 10], [0, 10], [0, 10]]",NaN,Al,0.0
1,[[ 0.71163319 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_LAMMPS_ELASTIC,109743c8-673c-43a3-b88e-99de5f0f232a,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-06,Al,0.0
2,[[ 0.71163319 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_LAMMPS_ELASTIC,918fc503-262d-4e4a-8052-379797c597a3,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-07,Al,0.0
3,[[ 0.71163319 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_LAMMPS_ELASTIC,98b5105e-0e0b-46cc-b527-0508dfe02c7c,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-05,Al,0.0
4,[[ 7.11633183e-01 3.87259148e-01 3.872591...,-3.360000,NaN,calc_LAMMPS_ELASTIC,d983d408-f820-43ec-a6a2-bb6b994f422a,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-08,Al,0.0
5,[[ 0.71163316 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_LAMMPS_ELASTIC,fe5c99bb-5ec2-48f2-9171-94f0b32a2380,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-04,Al,0.0
6,[[ 0.71163318 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_refine_structure_static,1be750ea-9f66-4000-86e0-d7d2f4ce60a7,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-04,Al,0.0
7,[[ 0.71163318 0.38725916 0.38725916 0. ...,-3.360000,NaN,calc_refine_structure_static,62d864a7-22c3-4d3a-9154-95828397ccc5,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-08,Al,0.0
8,[[ 0.71163319 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_refine_structure_static,68d92ed2-4d3f-4c19-9340-5a73c8b02b0c,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-05,Al,0.0
9,[[ 0.71163319 0.38725915 0.38725915 0. ...,-3.360000,NaN,calc_refine_structure_static,7741988b-267a-4f63-8f2a-7016a45c43e3,4.050000,NaN,4.050000,NaN,4.050000,...,1995--Angelo-J-E--Ni-Al-H,0b194551-81ba-45ba-b4e1-7e035729f11c,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",1.000000e-07,Al,0.0


## Generate plots

### Lattice constants

In [11]:
#Extract values
potentials = np.asarray(data.potential_id)
lattice_a = uc.get_in_units(data.final_a, 'angstrom')
lattice_b = uc.get_in_units(data.final_b, 'angstrom')
lattice_c = uc.get_in_units(data.final_c, 'angstrom')



#Initialize bokeh plot
a_plot = figure(plot_width=800, plot_height=600, x_range=list(np.unique(potentials)))
a_plot.xaxis.major_label_orientation = 1.2
a_plot.yaxis.axis_label = 'Lattice Parameter (A)'

#Add data
a_plot.circle(potentials, lattice_a, color='black')
a_plot.circle(potentials, lattice_b, color='black')
a_plot.circle(potentials, lattice_c, color='black')

#Add error bars
try:
    lattice_a_error = uc.get_in_units(data.final_a_err, 'angstrom')
    lattice_b_error = uc.get_in_units(data.final_b_err, 'angstrom')
    lattice_c_error = uc.get_in_units(data.final_c_err, 'angstrom')
    a_plot.segment(potentials, lattice_a-lattice_a_error, potentials, lattice_a+lattice_a_error, color='black')
    a_plot.segment(potentials, lattice_b-lattice_b_error, potentials, lattice_b+lattice_b_error, color='black')
    a_plot.segment(potentials, lattice_c-lattice_c_error, potentials, lattice_c+lattice_c_error, color='black')
except:
    pass
    
#Show plot
show(a_plot)

### Cohesive energy

In [12]:
#Extract values
potentials = np.asarray(data.potential_id)
E_coh = uc.get_in_units(data.E_cohesive, 'eV')


#Initialize bokeh plot
e_plot = figure(plot_width=800, plot_height=600, x_range=list(np.unique(potentials)))
e_plot.xaxis.major_label_orientation = 1.2
e_plot.yaxis.axis_label = 'Cohesive Energy (eV/atom)'

#Add data
e_plot.circle(potentials, E_coh, color='black')

#Add error bars
try:
    E_coh_error = uc.get_in_units(data.E_cohesive_err, 'eV')
    e_plot.segment(potentials, E_coh-E_coh_error, potentials, E_coh+E_coh_error, color='black')
except:
    pass

#Show plot
show(e_plot)

### Elastic constants

In [13]:
#Extract values
d = data[pd.notnull(data.C)]
potentials = np.asarray(d.potential_id)
c11 = []
c12 = []
c44 = []
for c in d.C:
    c11.append(uc.get_in_units((c.Cij[0,0] + c.Cij[1,1] + c.Cij[2,2]) / 3, 'GPa'))
    c12.append(uc.get_in_units((c.Cij[0,1] + c.Cij[0,2] + c.Cij[1,2]) / 3, 'GPa'))
    c44.append(uc.get_in_units((c.Cij[3,3] + c.Cij[4,4] + c.Cij[5,5]) / 3, 'GPa'))

#Initialize bokeh plot
c_plot = figure(plot_width=800, plot_height=600, x_range=list(np.unique(potentials)))
c_plot.xaxis.major_label_orientation = 1.2
c_plot.yaxis.axis_label = 'Elastic Constants (GPa)'

#Add data
c_plot.circle(potentials, c11, color='blue', legend='C11')
c_plot.circle(potentials, c12, color='red',  legend='C12')
c_plot.circle(potentials, c44, color='green',legend='C44')

#Show plot
show(c_plot)

In [14]:
save(vplot(a_plot, e_plot, c_plot), symbols+'.html', resources=Resources(), title=symbols)

In [15]:
if symbols == 'Al':
    pass
else:
    killit

In [16]:
#Extract values
p_data = data[data.potential_id == '2004--Zhou-X-W--Al']

lammps_elastic_data = p_data[p_data.calc_id=='calc_LAMMPS_ELASTIC']
lammps_elastic_strain = lammps_elastic_data.strain_range
lammps_elastic_c11 = []
for c in lammps_elastic_data.C:
    lammps_elastic_c11.append(uc.get_in_units((c.Cij[0,0] + c.Cij[1,1] + c.Cij[2,2]) / 3, 'GPa'))

relax_structure_data = p_data[p_data.calc_id=='calc_refine_structure_static']
relax_structure_strain = relax_structure_data.strain_range
relax_structure_c11 = []
for c in relax_structure_data.C:
    relax_structure_c11.append(uc.get_in_units((c.Cij[0,0] + c.Cij[1,1] + c.Cij[2,2]) / 3, 'GPa'))

#Initialize bokeh plot
c_plot = figure(plot_width=800, plot_height=600, x_axis_type='log', x_range=[1e-3, 1e-9], y_range=[-20,140])
#c_plot.xaxis.major_label_orientation = 1.2
c_plot.xaxis.axis_label = 'Strain range'
c_plot.yaxis.axis_label = 'C11 Elastic Constants (GPa)'

#Add data
c_plot.circle(lammps_elastic_strain, lammps_elastic_c11, color='blue', alpha=0.5, size=10, legend='LAMMPS_ELASTIC')
c_plot.circle(relax_structure_strain, relax_structure_c11, color='red', alpha=0.5, size=10,legend='relax_structure')

#Show plot
show(c_plot)

In [17]:
#Extract values
p_data = data[data.potential_id == '2004--Zhou-X-W--Al']

lammps_elastic_data = p_data[p_data.calc_id=='calc_LAMMPS_ELASTIC']
lammps_elastic_strain = lammps_elastic_data.strain_range
lammps_elastic_c12 = []
for c in lammps_elastic_data.C:
    lammps_elastic_c12.append(uc.get_in_units((c.Cij[0,1] + c.Cij[0,2] + c.Cij[1,2]) / 3, 'GPa'))

relax_structure_data = p_data[p_data.calc_id=='calc_refine_structure_static']
relax_structure_strain = relax_structure_data.strain_range
relax_structure_c12 = []
for c in relax_structure_data.C:
    relax_structure_c12.append(uc.get_in_units((c.Cij[0,1] + c.Cij[0,2] + c.Cij[1,2]) / 3, 'GPa'))

#Initialize bokeh plot
c_plot = figure(plot_width=800, plot_height=600, x_axis_type='log', x_range=[1e-3, 1e-9], y_range=[-20,140])
#c_plot.xaxis.major_label_orientation = 1.2
c_plot.xaxis.axis_label = 'Strain range'
c_plot.yaxis.axis_label = 'C12 Elastic Constants (GPa)'

#Add data
c_plot.circle(lammps_elastic_strain, lammps_elastic_c12, color='blue', alpha=0.5, size=10, legend='LAMMPS_ELASTIC')
c_plot.circle(relax_structure_strain, relax_structure_c12, color='red', alpha=0.5, size=10,legend='relax_structure')

#Show plot
show(c_plot)

In [18]:
#Extract values
p_data = data[data.potential_id == '2004--Zhou-X-W--Al']

lammps_elastic_data = p_data[p_data.calc_id=='calc_LAMMPS_ELASTIC']
lammps_elastic_strain = lammps_elastic_data.strain_range
lammps_elastic_c44 = []
for c in lammps_elastic_data.C:
    lammps_elastic_c44.append(uc.get_in_units((c.Cij[3,3] + c.Cij[4,4] + c.Cij[5,5]) / 3, 'GPa'))

relax_structure_data = p_data[p_data.calc_id=='calc_refine_structure_static']
relax_structure_strain = relax_structure_data.strain_range
relax_structure_c44 = []
for c in relax_structure_data.C:
    relax_structure_c44.append(uc.get_in_units((c.Cij[3,3] + c.Cij[4,4] + c.Cij[5,5]) / 3, 'GPa'))

#Initialize bokeh plot
c_plot = figure(plot_width=800, plot_height=600, x_axis_type='log', x_range=[1e-3, 1e-9], y_range=[-20,140])
#c_plot.xaxis.major_label_orientation = 1.2
c_plot.xaxis.axis_label = 'Strain range'
c_plot.yaxis.axis_label = 'C44 Elastic Constants (GPa)'

#Add data
c_plot.circle(lammps_elastic_strain, lammps_elastic_c44, color='blue', alpha=0.5, size=10, legend='LAMMPS_ELASTIC')
c_plot.circle(relax_structure_strain, relax_structure_c44, color='red', alpha=0.5, size=10,legend='relax_structure')

#Show plot
show(c_plot)

In [1]:
import os


In [5]:
calcs = os.listdir('C:/Users/lmh1/Documents/calculations/ipr/torun/3')
len(calcs)

35

In [4]:
print len(calcs)

6071
